In [16]:
import pandas as pd
df=pd.read_csv('/Users/sophiehu/Downloads/covid/merged.csv')
df

smalldf = df[df['title'].str.contains('terms')]
smalldf

sentiments = smalldf.copy()
sentiments.dropna(subset=['gram'], inplace=True)
sentiments

sentiments[['date', 'category']] = sentiments['title'].str.split('_', n=1, expand=True)

sentiments['date'] = pd.to_datetime(sentiments['date']).dt.date.astype(str)

sentiments

sentiments = sentiments.drop(columns=['title', 'category'])

sentiments["counts"] = pd.to_numeric(sentiments["counts"])
newdf = sentiments.groupby('date').apply(lambda x: x.nlargest(100, 'counts')).reset_index(drop=True)
newdf

,gram,counts,date
0,coronavirus,223843,2020-03-22
1,covid,67037,2020-03-22
2,19,64625,2020-03-22
3,covid19,56389,2020-03-22
4,people,28026,2020-03-22
...,...,...,...
20495,well,5898,2020-10-12
20496,du,5875,2020-10-12
20497,year,5838,2020-10-12
20498,oms,5803,2020-10-12


In [17]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [18]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

In [19]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [20]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


In [21]:

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)


In [22]:

text = newdf['gram'].tolist()

import torch
from transformers import RobertaTokenizer

In [23]:
# Convert the text list to a list of encoded token IDs


input_ids = []
for term in text:
    encoded_sent = tokenizer.encode(term, add_special_tokens=True)
    input_ids.append(encoded_sent)
    
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length to the length of the longest sequence in input_ids
max_len = max([len(seq) for seq in input_ids])

# Pad the sequences with 0s to make them all the same length
input_ids = pad_sequences(input_ids, maxlen=max_len, dtype="long", value=0, truncating="post", padding="post")


# Convert the list of input IDs to a tensor
input_ids = torch.tensor(input_ids)


In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np

output = []

for i in tqdm(input_ids):
    output.append(model(i.unsqueeze(0)))

 31%|███       | 6261/20500 [37:14<1:54:31,  2.07it/s] 

In [1]:
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

NameError: name 'output' is not defined

In [2]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

NameError: name 'np' is not defined